In [1]:
#importing the libraries needed 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint

In [2]:
#creating an empty list, so that we can append the values
movie_name, year,Director, time, rating ,metascore, votes, gross, description ,Stars ,Genre= ([] for i in range(11) )

In [3]:
#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,1000,100)

#the whole core of the script
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    
    #Random sleep interval so server does not kick/ban the IP
    sleep(randint(2,8))

    #calling one by one using for loop
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)

        year_of_release = store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
        year.append(year_of_release)

        runtime = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
        time.append(runtime)

        genre = store.p.find('span', class_ = 'genre').text.replace('\n', '')
        Genre.append(genre)

        rate = store.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
        rating.append(rate)

        meta  = store.find('span', class_ = 'metascore').text.replace(' ', '') if store.find('span', class_ = 'metascore') else '^^^^^^'
        metascore.append(meta)
        #since, gross and votes have same attributes, that's why we had created a common variable and then used indexing
        value = store.find_all('span', attrs = {'name': 'nv'})

        vote = value[0].text
        votes.append(vote)

        grosses = value[1].text if len(value) >1 else '*****'
        gross.append(grosses)

        # Description of the Movies -- Not explained in the Video, But you will figure it out. 
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
        description.append(description_)

        #Cast Details -- Scraping Director name and Stars -- Not explained in Video
        cast = store.find("p", class_ = '')
        cast = cast.text.replace('\n', '').split('|')
        cast = [x.strip() for x in cast]
        cast = [cast[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
        Director.append(cast[0])
        Stars.append([x.strip() for x in cast[1].split(",")])
    
#creating a dataframe 
movie_list = pd.DataFrame({ "Movie Name": movie_name,"Director":Director, "Year of Release" : year, \
                           "Watch Time": time,"Movie Rating": rating, "Meatscore of movie": metascore, \
                           "Votes" : votes, "Gross": gross,"MovieStars":Stars, "Description": description,\
                           "Genre":Genre  })

movie_list.head()

,Movie Name,Director,Year of Release,Watch Time,Movie Rating,Meatscore of movie,Votes,Gross,MovieStars,Description,Genre
0,The Shawshank Redemption,Frank Darabont,1994,142,9.3,82,"2,770,620",$28.34M,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will...","Over the course of several years, two convicts...",Drama
1,The Godfather,Francis Ford Coppola,1972,175,9.2,100,"1,928,706",$134.97M,"[Marlon Brando, Al Pacino, James Caan, Diane K...","Don Vito Corleone, head of a mafia family, dec...","Crime, Drama"
2,The Dark Knight,Christopher Nolan,2008,152,9.0,84,"2,745,314",$534.86M,"[Christian Bale, Heath Ledger, Aaron Eckhart, ...",When the menace known as the Joker wreaks havo...,"Action, Crime, Drama"
3,Schindler's List,Steven Spielberg,1993,195,9.0,95,"1,395,149",$96.90M,"[Liam Neeson, Ralph Fiennes, Ben Kingsley, Car...","In German-occupied Poland during World War II,...","Biography, Drama, History"
4,12 Angry Men,Sidney Lumet,1957,96,9.0,97,"821,806",$4.36M,"[Henry Fonda, Lee J. Cobb, Martin Balsam, John...",The jury in a New York City murder trial is fr...,"Crime, Drama"


In [4]:
movie_list['Gross'] = movie_list['Gross'].replace('*****','')
movie_list['Meatscore of movie'] = movie_list['Meatscore of movie'].replace('^^^^^^','')
movie_list['Gross'][movie_list['Gross'].str.contains('#')] = ''


In [7]:
#Saving as a csv file for further analysis
movie_list.to_csv('Documents\Movies.csv')